In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#ResNet block, Inception module, and DenseNet block functions
#(Same as in the previous example)

...

Ellipsis

In [ ]:
#ResNet block
def resnet_block(inputs, filters,kernel_size=3, strides=1, conv_shortcut=False):
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)

    if conv_shortcut:
        shortcut = layers.Conv2D(filters, 1, strides=strides)(inputs)
        shortcut = layers.BatchNormalization()(shortcut)
    else:
        shortcut = inputs
    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)
    return x


In [ ]:
#Inception module
def inception_module(inputs, filters):
    conv1x1 = layers.Conv2D(filters, 1, padding='same', activation='relu')(inputs)
    conv3x3 = layers.Conv2D(filters, 3, padding='same', activation='relu')(inputs)
    conv5x5 = layers.Conv2D(filters, 5, padding='same', activation='relu')(inputs)
    pool = layers.MaxPooling2D(3, strides=1, padding='same')(inputs)
    pool = layers.Conv2D(filters, 1, padding='same', activation='relu')(pool)
    output = layers.concatenate([conv1x1, conv3x3, conv5x5, pool], axis=-1)
    return output


In [ ]:
#DenseNet block
def densenet_block(inputs, growth_rate):
    x = layers.BatchNormalization()(inputs)
    x = layers.ReLU()(x)
    x = layers.Conv2D(growth_rate, 3, padding='same')(x)
    x = layers.concatenate([inputs, x], axis=-1)
    return x

In [ ]:
#Build the model
def build_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    #ResNet-like architecture
    x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    x = resnet_block(x, 64)
    x = resnet_block(x, 64)
    x = resnet_block(x, 128, strides=2, conv_shortcut=True)
    x = resnet_block(x, 128)

    #Inception module
    x = inception_module(x, 128)

    #DenseNet block
    x = densenet_block(x, 32)
    x = densenet_block(x, 32)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs)
    return model


In [ ]:
#Load and preprocess the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
#Build and compile the model
input_shape =  (32, 32, 3)
num_classes = 10
model = build_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Train the model
batch_size = 128
epochs = 10
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.4238 - loss: 1.6546 - val_accuracy: 0.2918 - val_loss: 2.0843
Epoch 2/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6343 - loss: 1.0149 - val_accuracy: 0.5936 - val_loss: 1.1312
Epoch 3/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.7178 - loss: 0.8017 - val_accuracy: 0.6424 - val_loss: 1.0322
Epoch 4/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.7698 - loss: 0.6481 - val_accuracy: 0.5360 - val_loss: 1.5100
Epoch 5/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8053 - loss: 0.5530 - val_accuracy: 0.6848 - val_loss: 0.9542
Epoch 6/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8391 - loss: 0.4561 - val_accuracy: 0.6368 - val_loss: 1.2269
Epoch 7/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.8690 - loss: 0.3692 - val_accuracy: 0.5942 - val_loss: 1.4952
Epoch 8/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8944 - loss: 0.3047 - val_acc

In [ ]:
#Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7297 - loss: 1.1086
Test accuracy: 0.7263000011444092
